In [2]:
from __future__ import unicode_literals
import tweepy
import csv
from tweepy import Cursor
from tweepy import OAuthHandler
from tweepy import API
import json
import pprint
import pathlib
pp = pprint.PrettyPrinter(indent=4)
                                                                     
import pprint as pp
import pandas as pd
import time

In [3]:
import yaml
with open(pathlib.Path.home().joinpath('api_login.yml'),'r') as fo:
    creds = yaml.safe_load(fo)
    consumer_key= creds['general']['development']['consumer_key']
    consumer_secret = creds['general']['development']['consumer_secret']
    access_token = creds['general']['development']['access_token']
    access_token_secret = creds['general']['development']['access_token_secret']
    youtube_key = creds['youtube']['key']
    news_key = creds['news']['key']
    botometer_key = creds['botometer']['key']

In [4]:
from twarc import Twarc
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# from requests_oauthlib import OAuth1
# 
# 
# base_twitter_url = "https://api.twitter.com/1.1/"
# 
# auth = OAuth1(client_key, client_secret, token, token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [80]:
def search_tweets(term: str, map_id = False, grouped: bool = True, limit = False):
    """
    This function uses Tweepy to fetch Twitter data
    Inputs:
        term: Search term
        map_id: Which map to merge data with
        grouped: Whether to merge with group data or not
        limit: How many 100s of search results to fetch
    Outputs:
        df: Dataframe of tweets returned from search
    """    
    alltweets = []
    new_tweets = api.search(f"{term}", count = 100,tweet_mode='extended')
    #print (new_tweets)

    #save most recent tweets
    alltweets.extend(new_tweets)
    #print (alltweets)
    #print (alltweets)

    #save the id of the oldest tweet less one
    if alltweets:
        oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    if limit == False:
        limit = 50000
    while len(new_tweets) > 0:
        if len(alltweets) >= limit:
            break
        try:
            print ("getting tweets before %s" %(oldest))
        except:
            return parse_tweepy_results(alltweets)
        #print (alltweets[-1].text)
        try:
            #all subsequent requests use the max_id param to prevent duplicates
            new_tweets = api.search(term, count = 100,max_id=oldest,tweet_mode='extended')
            #save most recent tweets
            alltweets.extend(new_tweets)

            if alltweets:
            #update the id of the oldest tweet less one
               # print (alltweets)
                oldest = alltweets[-1].id - 1

            print ("...%s tweets downloaded so far"%(len(alltweets)))
        except tweepy.TweepError as e:
            print(e.reason)
            time.sleep(60)

    return parse_tweepy_results(alltweets)


In [3]:
def parse_tweepy_results(alltweets: dict, map_id = False, grouped: bool = True) -> pd.DataFrame:
    tweets_features = []
    retweet_user_id = ''
    retweet_screen_name = ''
    retweet_tweet_id = ''
    retweet_count = 0
    favorite_count = 0
    for tweet in alltweets:
        tweet_dict = {}
        urls = []
        hashtags = []
        mentions = []
        try:
            text_field = tweet.full_text
        except:
            text_field = tweet.text
        if 'quoted_status' in tweet._json.keys():
#             print('QT')
#             display(tweet)
            try:
                retweet_text = tweet.quoted_status.full_text
                try:
                    text = tweet.full_text
                except:
                    text = tweet.text
                retweet = True
                retweet_user_id = tweet.quoted_status.user.id_str
                retweet_screen_name = tweet.quoted_status.user.screen_name
                retweet_tweet_id = tweet.quoted_status.id_str
                retweet_count = tweet.retweet_count
                favorite_count = tweet.favorite_count
                
#                 urls = []
                for u in tweet.quoted_status.entities["urls"]:
                    urls.append(u["expanded_url"])

#                 hashtags = []
                for h in tweet.quoted_status.entities["hashtags"]:
                    hashtags.append(h['text'])

#                 mentions = []    
                for m in tweet.quoted_status.entities["user_mentions"]:
                    mentions.append(m['screen_name'])
                
                retweet = True
                quote_tweet = True
            except AttributeError:
                retweet = True
                quote_tweet = True
                text = tweet.full_text#.encode("utf-8")
                
        
        elif str(text_field)[:3] == "RT ":
#             print('RT')
#             display(tweet)
            try:
                try:
                    retweet_text = tweet.retweeted_status.full_text
                except:
                    retweet_text = tweet.retweeted_status.text
                try:
                    text = tweet.full_text
                except:
                    text = tweet.text
                retweet = True
                retweet_user_id = tweet.retweeted_status.user.id_str
                retweet_screen_name = tweet.retweeted_status.user.screen_name
                retweet_tweet_id = tweet.retweeted_status.id_str
                retweet_count = tweet.retweet_count
                favorite_count = tweet.favorite_count
                
#                 urls = []
                for u in tweet.retweeted_status.entities["urls"]:
                    urls.append(u["expanded_url"])

#                 hashtags = []
                for h in tweet.retweeted_status.entities["hashtags"]:
                    hashtags.append(h['text'])

#                 mentions = []    
                for m in tweet.retweeted_status.entities["user_mentions"]:
                    mentions.append(m['screen_name'])

                retweet = True
                quote_tweet = False
            
            except AttributeError:
                retweet = True
                quote_tweet = False
                try:
                    text = tweet.full_text#.encode("utf-8")
                except:
                    text = tweet.text

        else:
            try:
                text = tweet.full_text
            except:
                text = tweet.text#.encode("utf-8") #indented this here
            retweet = False
            quote_tweet = False
            retweet_text = None
            retweet_user_id = None
            retweet_screen_name = None
            retweet_tweet_id = None

            retweet_count = tweet.retweet_count
            favorite_count = tweet.favorite_count

#         urls = []
        for u in tweet.entities["urls"]:
            urls.append(u["expanded_url"])

#         hashtags = []
        for h in tweet.entities["hashtags"]:
            hashtags.append(h['text'])

#         mentions = []    
        for m in tweet.entities["user_mentions"]:
            mentions.append(m['screen_name'])
    
        tweet_dict["retweet_user_id"] = retweet_user_id
        tweet_dict["retweet_screen_name"] = retweet_screen_name
        tweet_dict["retweet_tweet_id"] = retweet_tweet_id 
        tweet_dict["user_id"] = tweet.user.id_str
        tweet_dict["retweet"] = retweet
        tweet_dict["text"] = text
        tweet_dict['retweet_text'] = retweet_text
        tweet_dict["author"] = tweet.author.name
        tweet_dict["screen_name"] = tweet.author.screen_name
        tweet_dict["tweet_id"] = tweet.id_str
        tweet_dict["time"] = tweet.created_at
        tweet_dict["mentions"] = mentions
        tweet_dict["hashtags"] = hashtags
        tweet_dict["urls"] = urls
        tweet_dict['retweet_count'] = retweet_count
        tweet_dict['favorite_count'] = favorite_count
        tweet_dict['source'] = tweet.source
        tweet_dict['is_retweet'] = retweet
        tweet_dict['is_quote_tweet'] = quote_tweet
        tweets_features.append(tweet_dict)

        #also need author id so can recreated url, might also be an easier way

    #transform the tweepy tweets into a 2D array that will populate the csv	
    # outtweets = [[tweet.author.name, tweet.id_str, tweet.created_at, tweet.full_text.encode("utf-8")] for tweet in alltweets]

    df = pd.DataFrame(tweets_features)
    
    if map_id:
        nodes = get_node_data(map_id,grouped)
        df = pd.merge(nodes,df)
        
    print('...Done!')
    return df

In [82]:
def get_full_tweets(tweet_id_list: list):
    print('...Getting {} tweets'.format(len(tweet_id_list)))
#     tweet_id_list = [1246082527544180736,1246086302178537473]
    result_tweets = []
    for i in range(0,len(tweet_id_list),100):
        try:
            result_tweets.extend(api.statuses_lookup(tweet_id_list[i:i+100],tweet_mode = 'extended'))
        except:
            continue
    return parse_tweepy_results(result_tweets)

In [4]:
def get_all_users(node_id_list: list,field: str = 'id_str'):
    node_id_list = [str(x) for x in node_id_list]
    wanted_list = node_id_list
    filename = '/Users/avneeshchandra/Downloads/all_screennames.csv'
    print('...Getting {} users'.format(len(node_id_list)))
    screennames_df = stringify_ids(pd.read_csv(filename))
    try:
        node_id_list = list(set(node_id_list).difference(screennames_df[field].unique().tolist()))
    except:
        pass
    print(f'...Only fetching {len(node_id_list)} new user(s)')
    if len(node_id_list) > 0:
        result_users = []
        for i in range(0,len(node_id_list),100):
            try:
                if field == 'screen_name':
                        result_users.extend(api.lookup_users(screen_names = node_id_list[i:i+100]))
                else:
                    result_users.extend(api.lookup_users(node_id_list[i:i+100]))
            except:
                continue
        screennames_df = screennames_df.append(stringify_ids(pd.DataFrame([x._json for x in result_users])))
        screennames_df.to_csv(filename,index = False)

    screennames_df = screennames_df[screennames_df[field].isin(wanted_list)].drop_duplicates('id_str')
    return screennames_df

In [5]:
def get_list_members(list_ids: list):
    members = []
    for i in list_ids:
        for page in tweepy.Cursor(api.list_members, list_id = i).items():
            members.append(page)
    temp_df = pd.DataFrame([x._json for x in members])
    return temp_df

In [6]:
def graphika_segments(map_id: str = '3108'):
    r = re.get(f'http://api.graphika.com/maps/{map_id}/groups', auth=(username, pswd))
    groups_df = pd.DataFrame(r.json())
    r = re.get(f'http://api.graphika.com/maps/{map_id}/clusters', auth=(username, pswd))
    clusters_df = pd.DataFrame(r.json())
    segments_df = clusters_df[['name','group_no','cluster_no']].rename(columns = {'name':'cluster_name'})\
        .merge(groups_df[['group_no','name']].rename(columns = {'name':'group_name'}))
    segments_df = segments_df[['group_no','group_name','cluster_no','cluster_name']].sort_values('group_no')
#     display(segments_df)
    return segments_df